# Tiền xử lý dữ liệu

## Tại sao cần phải tiền xử lý dữ liệu
Tiền xử lý dữ liệu là một phần quan trọng trong quy trình phân tích dữ liệu và xây dựng mô hình máy học, giúp cải thiện chất lượng và hiệu suất của mô hình cuối cùng.

## Mục tiêu
Ở phần này, dữ liệu chúng ta thu được là dữ liệu dạng **văn bản**. Tuy nhiên các mô hình học máy chỉ có thể "học" khi dữ liệu là dạng **số thực** nên trong phần này, việc quan trọng nhất đó chính là *Làm thế nào để chuyển dữ liệu sang dạng số thực?* 

## Import các thư viện cần dùng

In [2]:
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import plotly.express as px
import matplotlib.pyplot as plt
from googletrans import Translator

### Đọc dữ liệu từ thư mục data/
Vì dữ liệu được lưu dưới dạng JSON, nên chúng ta có thể sử dụng trực tiếp phương thức `read_json()` trong thư viện `pandas`

In [3]:
data = pd.read_json("../data/dantri.json")
data.head()

,title,category,content
0,"Thi hành án dứt điểm những vụ lớn, phức tạp li...",Xã hội,"Ông Nguyễn Văn Sơn, Phó Tổng cục trưởng Tổng c..."
1,"Món quà của Thủ tướng dành cho những người ""kh...",Xã hội,"Sáng 12/2 (mùng 3 Tết Giáp Thìn), Thủ tướng Ph..."
2,Thủ tướng thị sát đường sắt Nhổn - ga Hà Nội: ...,Xã hội,"Sáng 12/2 (mùng 3 Tết Giáp Thìn), Thủ tướng Ph..."
3,Đại sứ Saudi Arabia: Mong muốn Việt Nam sớm vư...,Xã hội,Thủ tướng Phạm Minh Chính tiếp Đại sứ Saudi Ar...
4,"Tổng Thanh tra Chính phủ nói về giải pháp ""trị...",Xã hội,Tổng Thanh tra Chính phủ Đoàn Hồng Phong vừa c...


In [4]:
print(f"Dữ liệu gồm {data.shape[0]} theo dõi và {data.shape[1]} thuộc tính")

Dữ liệu gồm 13931 theo dõi và 3 thuộc tính


### Loại trừ các dữ liệu NaN

In [5]:
data.isnull().sum()

title       0
category    0
content     0
dtype: int64

Dữ liệu trên, không có theo dõi nào có thuộc tính bị NaN hoặc rỗng, do đó, ở phần này chúng ta không cần xử lý phần việc loại trừ các giá trị bị rỗng

> **Lưu ý**: Mục tiêu của bài toán là dựa vào nội dung (**content**) để phân loại dữ liệu thuộc hạng mục nào (**category**). Do đó, có hai lí do để chúng ta chỉ chọn cột **category** và **content**. Thứ nhất, là do giá trị cột **title** thường quá ngắn để có thể phân loại và thứ hai, chúng ta sẽ dựa vào nội dung (**content**) để phân loại.

In [6]:
data = data[["content", "category"]]
data.head()

,content,category
0,"Ông Nguyễn Văn Sơn, Phó Tổng cục trưởng Tổng c...",Xã hội
1,"Sáng 12/2 (mùng 3 Tết Giáp Thìn), Thủ tướng Ph...",Xã hội
2,"Sáng 12/2 (mùng 3 Tết Giáp Thìn), Thủ tướng Ph...",Xã hội
3,Thủ tướng Phạm Minh Chính tiếp Đại sứ Saudi Ar...,Xã hội
4,Tổng Thanh tra Chính phủ Đoàn Hồng Phong vừa c...,Xã hội


In [7]:
count = data["category"].value_counts()
num_of_post_each_topic = pd.DataFrame({
    'category': count.index,
    'count': count.values
})

In [8]:
count.index

Index(['Thể thao', 'Giáo dục', 'Sức khỏe', 'Bất động sản', 'Kinh doanh',
       'Văn hóa', 'Xã hội', 'Xe ++', 'Sức mạnh số', 'Thế giới', 'Việc làm',
       'Giải trí', 'Nhân ái', 'An sinh', 'Pháp luật'],
      dtype='object', name='category')

In [9]:
num_of_post_each_topic

,category,count
0,Thể thao,1497
1,Giáo dục,1414
2,Sức khỏe,1270
3,Bất động sản,1229
4,Kinh doanh,1102
5,Văn hóa,1044
6,Xã hội,967
7,Xe ++,910
8,Sức mạnh số,800
9,Thế giới,759


In [10]:
fig = px.histogram(
    num_of_post_each_topic,
    x='category',
    y='count',
    color='category',
    title="Số lượng bài báo trên từng hạng mục",
    color_discrete_sequence=px.colors.qualitative.Set1,
)
fig.update_yaxes(showgrid=True)  # the y-axis is in dollars
fig.update_layout(  # customize font and legend orientation & position
    font_family="Arial",
    legend=dict(
        title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
    ),
    yaxis_title="Số lượng bài báo",
    xaxis_title="Hạng mục",
)

fig.add_shape(  # add a horizontal "target" line
    type="line",
    line_color="#16FF32",
    line_width=3,
    opacity=1,
    line_dash="dot",
    x0=0,
    x1=1,
    xref="paper",
    y0=929,
    y1=929,
    yref="y",
)

fig.add_annotation(  # add a text callout with arrow
    text="Số lượng ít nhất", x="Pháp luật", y=500, arrowhead=1, showarrow=True
)


fig.show()

e:\PKA-Project\BTL-KPDL\.venv\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


Dựa vào hình trên, nhận thấy rằng, số lượng bài báo thu được ở hạng mục `Thể thao` là nhiều nhất với gần 1k5 bài, trong khi đó, số lượng bài báo thu được ở hạng mục `Pháp luật` là ít nhất với khoảng 400 bài

> Nhìn chung, dữ liệu thu được khoảng 14k bài báo, chưa đủ nhiều đến mức đánh giá bài toán là **Imbalanced Data** nhưng chúng ta có thể xét vào bài toán trên và đánh giá nếu hiệu suất ổn hơn

### Kịch bản xử lý
Để đa dạng trong việc so sánh hiệu suất, chúng tôi đề xuất hai kịch bản thự nghiệm như sau:


*   **Kịch bản 1**: Sử dụng thư viện `googletrans`, một giao diện Python cho Google Translate API để dịch đoạn trên thành tiếng Anh và sau đó sẽ xây dựng mô hình học máy
*   **Kịch bản 2**: Xử lý trực tiếp đoạn văn bản trên và xây dựng mô hình học máy



#### Kịch bản 1: Dịch các đoạn trên sang tiếng Anh và xây dựng mô hình học máy

Vì nội dung của một bài báo khá dài, nên việc dịch cả đoạn một cách trực tiếp có thể dẫn đến việc mất mát thông tin, điều này là không tốt, vì dữ liệu đã phải dịch qua một ngôn ngữ khác và sau đó xử lý trước khi đưa vào mô hình vì vậy việc tối ưu sự mất mát thông tin ngay từ bước đầu tiên sẽ tốt hơn nhiều.

In [11]:
nltk.sent_tokenize(data["content"][0])

['Ông Nguyễn Văn Sơn, Phó Tổng cục trưởng Tổng cục Thi hành án dân sự (Bộ Tư pháp) vừa ký ban hành kế hoạch công tác năm 2024 của Tổ công tác chỉ đạo xử lý các vụ việc liên quan đến hoạt động tín dụng, ngân hàng (Tổ xử lý nợ xấu).',
 'Tổ xử lý nợ xấu có trách nhiệm phối hợp với Vụ Pháp chế (Ngân hàng Nhà nước), Hiệp hội Ngân hàng tổ chức tọa đàm, đối thoại giữa các cơ quan thi hành án với tổ chức tín dụng, Công ty quản lý tài sản (VAMC) để trao đổi, tháo gỡ khó khăn, vướng mắc liên quan đến các vụ việc thi hành án tín dụng, ngân hàng.',
 'Trong đó, tập trung rà soát, tổng hợp các vụ việc trên cơ sở thông báo kết luận kiểm tra các năm 2022, 2023 hoặc đề nghị của Hiệp hội Ngân hàng, các tổ chức tín dụng, VAMC.',
 '"Phối hợp chỉ đạo tổ chức thi hành dứt điểm đối với những vụ việc có điều kiện, nhất là những đơn vị có nhiều vụ việc lớn, phức tạp liên quan đến các tổ chức tín dụng như Agribank, Vietcombank, VPBank, BIDV, Vietinbank", lãnh đạo Tổng cục Thi hành án dân sự nêu rõ.',
 'Ông Nguy

In [21]:
def doc_to_line(tokens: list[str]):
    return ' '.join(tokens)

# Dịch đoạn văn sang tiếng Anh
def translate_from_vi_to_en(content: str):
    translator = Translator(
        user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
        raise_exception=False,
    )
    sentences = nltk.sent_tokenize(content)
    translated_text = [
        translator.translate(sentence, src="vi", dest="en").text for sentence in sentences
    ]
    return doc_to_line(translated_text)

In [22]:
data['content_en'] = data['content'].apply(translate_from_vi_to_en)

ReadTimeout: The read operation timed out